# Sistem Rekomendasi Buku

###Proyek Akhir Machine Learning Terapan

* Nama : Safiira Rahmah Linisa
* SIB ID : M248Y0537
* SIB Group : M02

Dowload data dari kaggle kemudian unzip file yang sudah didowload.

In [1]:
! chmod 600 /content/kaggle.json

In [2]:
! KAGGLE_CONFIG_DIR=/content/ kaggle datasets download -d justinnguyen0x0x/best-books-of-the-21st-century-dataset

  0% 0.00/1.06M [00:00<?, ?B/s]
100% 1.06M/1.06M [00:00<00:00, 131MB/s]


##Data Understanding

Import terlebih dahulu library yang dibutuhkan

In [3]:
# Import library
import pandas as pd
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

In [4]:
import zipfile
zip_file = zipfile.ZipFile('/content/best-books-of-the-21st-century-dataset.zip')
zip_file.extractall('/tmp/')
zip_file.close()

###Data Loading

Membaca datset menggunakan fungsi pandas, kemudian melihat isi dataset dengan memanggil variabel yang berisi dataset tersebut.

In [5]:
book = pd.read_csv('/tmp/Best_Book_21st.csv')
book

,id,title,series,author,book_link,genre,date_published,publisher,num_of_page,lang,review_count,rating_count,rate,award
0,0,The Martian,\n (The Martian #1)\n,Andy Weir,https://www.goodreads.com/book/show/18007564-t...,"Science Fiction,Fiction,Audiobook,Adventure,Sp...",February 11th 2014,"Crown\n\n ,",384.0,English,"77,156","919,781",4.40,Kurd-Laßwitz-Preis for Bestes ausländisches We...
1,1,Under the Banner of Heaven: A Story of Violent...,NaN,Jon Krakauer,https://www.goodreads.com/book/show/10847.Unde...,"Nonfiction,Religion,History,Crime,True Crime,M...",2004,"Pan MacMillan\n\n ,",400.0,English,"10,981","173,895",3.98,Colorado Book Award for General Nonfiction (2004)
2,2,Cutting for Stone,NaN,Abraham Verghese,https://www.goodreads.com/book/show/3591262-cu...,"Fiction,Historical,Historical Fiction,Cultural...",February 3rd 2009,"Alfred A. Knopf\n\n ,",541.0,English,"26,253","353,365",4.29,"PEN/Hemingway Foundation Award Nominee (2010),..."
3,3,We Need to Talk About Kevin,NaN,Lionel Shriver,https://www.goodreads.com/book/show/80660.We_N...,"Fiction,Contemporary,Thriller,Horror,Mystery,C...",July 3rd 2006,"Harper Perennial\n\n ,",400.0,English,"14,787","187,639",4.04,Orange Prize for Fiction (2005)
4,4,The Immortal Life of Henrietta Lacks,NaN,Rebecca Skloot,https://www.goodreads.com/book/show/6493208-th...,"Nonfiction,Science,History,Biography,Health,Me...",February 2nd 2010,"Crown Publishing Group\n\n ,",370.0,English,"36,574","646,155",4.07,Ambassador Book Award for American Studies (20...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10013,10013,"Edge of Passion (Stealth Guardians, #1)",\n (Stealth Guardians #1)\n,Tina Folsom,https://www.goodreads.com/book/show/13516444-e...,"Fantasy,Paranormal,Romance,Romance,Paranormal ...",March 7th 2012,NaN,NaN,English,102,"1,011",4.01,NaN
10014,10014,Tales from a Not-So-Popular Party Girl (Dork D...,\n (Dork Diaries #2)\n,Rachel Renée Russell,https://www.goodreads.com/book/show/8274537-ta...,"Childrens,Middle Grade,Realistic Fiction,Child...",June 8th 2010,Aladdin,288.0,English,"1,480","54,285",4.24,NaN
10015,10015,Bottoms,NaN,Tanja Kirschner,https://www.goodreads.com/book/show/2024071.Bo...,NaN,September 1st 2005,"NorthSouth\n\n ,",24.0,English,5,11,3.55,NaN
10016,10016,The Wedding Dress,NaN,Gary E. Parker,https://www.goodreads.com/book/show/783968.The...,NaN,January 9th 2002,"Victor\n\n ,",123.0,English,2,23,3.61,NaN


##**Data Preprocessing**

Melihat informasi pada data

In [6]:
book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10018 entries, 0 to 10017
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              10018 non-null  int64  
 1   title           10018 non-null  object 
 2   series          4341 non-null   object 
 3   author          10003 non-null  object 
 4   book_link       10018 non-null  object 
 5   genre           9019 non-null   object 
 6   date_published  9970 non-null   object 
 7   publisher       9624 non-null   object 
 8   num_of_page     9737 non-null   float64
 9   lang            9503 non-null   object 
 10  review_count    10003 non-null  object 
 11  rating_count    10003 non-null  object 
 12  rate            10003 non-null  float64
 13  award           4024 non-null   object 
dtypes: float64(2), int64(1), object(11)
memory usage: 1.1+ MB


Melihat informasi teratas

In [7]:
book.head()

,id,title,series,author,book_link,genre,date_published,publisher,num_of_page,lang,review_count,rating_count,rate,award
0,0,The Martian,\n (The Martian #1)\n,Andy Weir,https://www.goodreads.com/book/show/18007564-t...,"Science Fiction,Fiction,Audiobook,Adventure,Sp...",February 11th 2014,"Crown\n\n ,",384.0,English,"77,156","919,781",4.40,Kurd-Laßwitz-Preis for Bestes ausländisches We...
1,1,Under the Banner of Heaven: A Story of Violent...,NaN,Jon Krakauer,https://www.goodreads.com/book/show/10847.Unde...,"Nonfiction,Religion,History,Crime,True Crime,M...",2004,"Pan MacMillan\n\n ,",400.0,English,"10,981","173,895",3.98,Colorado Book Award for General Nonfiction (2004)
2,2,Cutting for Stone,NaN,Abraham Verghese,https://www.goodreads.com/book/show/3591262-cu...,"Fiction,Historical,Historical Fiction,Cultural...",February 3rd 2009,"Alfred A. Knopf\n\n ,",541.0,English,"26,253","353,365",4.29,"PEN/Hemingway Foundation Award Nominee (2010),..."
3,3,We Need to Talk About Kevin,NaN,Lionel Shriver,https://www.goodreads.com/book/show/80660.We_N...,"Fiction,Contemporary,Thriller,Horror,Mystery,C...",July 3rd 2006,"Harper Perennial\n\n ,",400.0,English,"14,787","187,639",4.04,Orange Prize for Fiction (2005)
4,4,The Immortal Life of Henrietta Lacks,NaN,Rebecca Skloot,https://www.goodreads.com/book/show/6493208-th...,"Nonfiction,Science,History,Biography,Health,Me...",February 2nd 2010,"Crown Publishing Group\n\n ,",370.0,English,"36,574","646,155",4.07,Ambassador Book Award for American Studies (20...


In [8]:
book.describe()

,id,num_of_page,rate
count,10018.000000,9737.000000,10003.000000
mean,5008.500000,356.103112,3.987817
std,2892.091832,201.543908,0.342731
min,0.000000,0.000000,0.000000
25%,2504.250000,256.000000,3.790000
50%,5008.500000,336.000000,3.990000
75%,7512.750000,420.000000,4.190000
max,10017.000000,4167.000000,5.000000


##**Data Preparation**

Memeriksa Missing Value Pada dataset

In [9]:
book.isnull().sum()

id                   0
title                0
series            5677
author              15
book_link            0
genre              999
date_published      48
publisher          394
num_of_page        281
lang               515
review_count        15
rating_count        15
rate                15
award             5994
dtype: int64

Dari hasil diatas dapat dilihat bahwa pada dataset terdapat banyak variabel yang memiliki missing value.

Selanjutnya menghapus variabel yang tidak diperlukan serta variabel mengandung banyak missing value dengan fungsi drop.

In [10]:
book.drop(['series', 'book_link', 'date_published', 'publisher', 'num_of_page', 'lang', 'rating_count', 'award'], axis = 1, inplace = True)

In [11]:
book

,id,title,author,genre,review_count,rate
0,0,The Martian,Andy Weir,"Science Fiction,Fiction,Audiobook,Adventure,Sp...","77,156",4.40
1,1,Under the Banner of Heaven: A Story of Violent...,Jon Krakauer,"Nonfiction,Religion,History,Crime,True Crime,M...","10,981",3.98
2,2,Cutting for Stone,Abraham Verghese,"Fiction,Historical,Historical Fiction,Cultural...","26,253",4.29
3,3,We Need to Talk About Kevin,Lionel Shriver,"Fiction,Contemporary,Thriller,Horror,Mystery,C...","14,787",4.04
4,4,The Immortal Life of Henrietta Lacks,Rebecca Skloot,"Nonfiction,Science,History,Biography,Health,Me...","36,574",4.07
...,...,...,...,...,...,...
10013,10013,"Edge of Passion (Stealth Guardians, #1)",Tina Folsom,"Fantasy,Paranormal,Romance,Romance,Paranormal ...",102,4.01
10014,10014,Tales from a Not-So-Popular Party Girl (Dork D...,Rachel Renée Russell,"Childrens,Middle Grade,Realistic Fiction,Child...","1,480",4.24
10015,10015,Bottoms,Tanja Kirschner,NaN,5,3.55
10016,10016,The Wedding Dress,Gary E. Parker,NaN,2,3.61


In [12]:
len(book)

10018

In [13]:
book.isnull().sum()

id                0
title             0
author           15
genre           999
review_count     15
rate             15
dtype: int64

Menghapus missing value pada data

In [14]:
book_clean = book.dropna()
book_clean

,id,title,author,genre,review_count,rate
0,0,The Martian,Andy Weir,"Science Fiction,Fiction,Audiobook,Adventure,Sp...","77,156",4.40
1,1,Under the Banner of Heaven: A Story of Violent...,Jon Krakauer,"Nonfiction,Religion,History,Crime,True Crime,M...","10,981",3.98
2,2,Cutting for Stone,Abraham Verghese,"Fiction,Historical,Historical Fiction,Cultural...","26,253",4.29
3,3,We Need to Talk About Kevin,Lionel Shriver,"Fiction,Contemporary,Thriller,Horror,Mystery,C...","14,787",4.04
4,4,The Immortal Life of Henrietta Lacks,Rebecca Skloot,"Nonfiction,Science,History,Biography,Health,Me...","36,574",4.07
...,...,...,...,...,...,...
10011,10011,Time's Echo,Pamela Hartshorne,"Historical,Historical Fiction,Science Fiction,...",193,3.88
10012,10012,"The Great Good Thing (The Sylvie Cycle, #1)",Roderick Townley,"Fantasy,Fiction,Young Adult,Childrens,Children...",329,3.92
10013,10013,"Edge of Passion (Stealth Guardians, #1)",Tina Folsom,"Fantasy,Paranormal,Romance,Romance,Paranormal ...",102,4.01
10014,10014,Tales from a Not-So-Popular Party Girl (Dork D...,Rachel Renée Russell,"Childrens,Middle Grade,Realistic Fiction,Child...","1,480",4.24


In [15]:
book_clean.isnull().sum()

id              0
title           0
author          0
genre           0
review_count    0
rate            0
dtype: int64

In [16]:
len(book_clean)

9019

In [17]:
# Mengurutkan buku berdasarkan id kemudian memasukkannya ke dalam variabel fix_book
fix_book = book_clean.sort_values('id', ascending=True)
fix_book

,id,title,author,genre,review_count,rate
0,0,The Martian,Andy Weir,"Science Fiction,Fiction,Audiobook,Adventure,Sp...","77,156",4.40
1,1,Under the Banner of Heaven: A Story of Violent...,Jon Krakauer,"Nonfiction,Religion,History,Crime,True Crime,M...","10,981",3.98
2,2,Cutting for Stone,Abraham Verghese,"Fiction,Historical,Historical Fiction,Cultural...","26,253",4.29
3,3,We Need to Talk About Kevin,Lionel Shriver,"Fiction,Contemporary,Thriller,Horror,Mystery,C...","14,787",4.04
4,4,The Immortal Life of Henrietta Lacks,Rebecca Skloot,"Nonfiction,Science,History,Biography,Health,Me...","36,574",4.07
...,...,...,...,...,...,...
10011,10011,Time's Echo,Pamela Hartshorne,"Historical,Historical Fiction,Science Fiction,...",193,3.88
10012,10012,"The Great Good Thing (The Sylvie Cycle, #1)",Roderick Townley,"Fantasy,Fiction,Young Adult,Childrens,Children...",329,3.92
10013,10013,"Edge of Passion (Stealth Guardians, #1)",Tina Folsom,"Fantasy,Paranormal,Romance,Romance,Paranormal ...",102,4.01
10014,10014,Tales from a Not-So-Popular Party Girl (Dork D...,Rachel Renée Russell,"Childrens,Middle Grade,Realistic Fiction,Child...","1,480",4.24


In [18]:
# Mengecek berapa jumlah fix_book
len(fix_book.id.unique())

9019

In [19]:
# Membuat variabel preparation yang berisi dataframe fix_book kemudian mengurutkan berdasarkan id
preparation = fix_book
preparation.sort_values('id')

,id,title,author,genre,review_count,rate
0,0,The Martian,Andy Weir,"Science Fiction,Fiction,Audiobook,Adventure,Sp...","77,156",4.40
1,1,Under the Banner of Heaven: A Story of Violent...,Jon Krakauer,"Nonfiction,Religion,History,Crime,True Crime,M...","10,981",3.98
2,2,Cutting for Stone,Abraham Verghese,"Fiction,Historical,Historical Fiction,Cultural...","26,253",4.29
3,3,We Need to Talk About Kevin,Lionel Shriver,"Fiction,Contemporary,Thriller,Horror,Mystery,C...","14,787",4.04
4,4,The Immortal Life of Henrietta Lacks,Rebecca Skloot,"Nonfiction,Science,History,Biography,Health,Me...","36,574",4.07
...,...,...,...,...,...,...
10011,10011,Time's Echo,Pamela Hartshorne,"Historical,Historical Fiction,Science Fiction,...",193,3.88
10012,10012,"The Great Good Thing (The Sylvie Cycle, #1)",Roderick Townley,"Fantasy,Fiction,Young Adult,Childrens,Children...",329,3.92
10013,10013,"Edge of Passion (Stealth Guardians, #1)",Tina Folsom,"Fantasy,Paranormal,Romance,Romance,Paranormal ...",102,4.01
10014,10014,Tales from a Not-So-Popular Party Girl (Dork D...,Rachel Renée Russell,"Childrens,Middle Grade,Realistic Fiction,Child...","1,480",4.24


**Menghapus Data Duplikat**

In [20]:
# Membuang data duplikat pada variabel preparation
preparation = preparation.drop_duplicates('title')
preparation

,id,title,author,genre,review_count,rate
0,0,The Martian,Andy Weir,"Science Fiction,Fiction,Audiobook,Adventure,Sp...","77,156",4.40
1,1,Under the Banner of Heaven: A Story of Violent...,Jon Krakauer,"Nonfiction,Religion,History,Crime,True Crime,M...","10,981",3.98
2,2,Cutting for Stone,Abraham Verghese,"Fiction,Historical,Historical Fiction,Cultural...","26,253",4.29
3,3,We Need to Talk About Kevin,Lionel Shriver,"Fiction,Contemporary,Thriller,Horror,Mystery,C...","14,787",4.04
4,4,The Immortal Life of Henrietta Lacks,Rebecca Skloot,"Nonfiction,Science,History,Biography,Health,Me...","36,574",4.07
...,...,...,...,...,...,...
10011,10011,Time's Echo,Pamela Hartshorne,"Historical,Historical Fiction,Science Fiction,...",193,3.88
10012,10012,"The Great Good Thing (The Sylvie Cycle, #1)",Roderick Townley,"Fantasy,Fiction,Young Adult,Childrens,Children...",329,3.92
10013,10013,"Edge of Passion (Stealth Guardians, #1)",Tina Folsom,"Fantasy,Paranormal,Romance,Romance,Paranormal ...",102,4.01
10014,10014,Tales from a Not-So-Popular Party Girl (Dork D...,Rachel Renée Russell,"Childrens,Middle Grade,Realistic Fiction,Child...","1,480",4.24


Mengkonversi data series menjadi list menggunakan fungsi tolist() dari library numpy.

In [21]:
# Mengonversi data series ‘id’ menjadi dalam bentuk list
book_id = preparation['id'].tolist()
 
# Mengonversi data series ‘title’ menjadi dalam bentuk list
book_title = preparation['title'].tolist()
 
# Mengonversi data series ‘author’ menjadi dalam bentuk list
book_author = preparation['author'].tolist()

# Mengonversi data series ‘genre’ menjadi dalam bentuk list
book_genre = preparation['genre'].tolist()
 
 # Mengonversi data series ‘review_count’ menjadi dalam bentuk list
book_review_count = preparation['review_count'].tolist()
 
 # Mengonversi data series ‘rate’ menjadi dalam bentuk list
book_rate = preparation['rate'].tolist()
 
 
print(len(book_id))
print(len(book_title))
print(len(book_author))
print(len(book_genre))
print(len(book_review_count))
print(len(book_rate))


8140
8140
8140
8140
8140
8140


In [23]:
# Membuat dictionary untuk data ‘id’, ‘title’, ‘author’, ‘genre’, ‘review’ dan ‘rate’.
book_new = pd.DataFrame({
    'id': book_id,
    'title': book_title,
    'author': book_author,
    'genre': book_genre,
    'review_count': book_review_count,
    'rate': book_rate
})
book_new

,id,title,author,genre,review_count,rate
0,0,The Martian,Andy Weir,"Science Fiction,Fiction,Audiobook,Adventure,Sp...","77,156",4.40
1,1,Under the Banner of Heaven: A Story of Violent...,Jon Krakauer,"Nonfiction,Religion,History,Crime,True Crime,M...","10,981",3.98
2,2,Cutting for Stone,Abraham Verghese,"Fiction,Historical,Historical Fiction,Cultural...","26,253",4.29
3,3,We Need to Talk About Kevin,Lionel Shriver,"Fiction,Contemporary,Thriller,Horror,Mystery,C...","14,787",4.04
4,4,The Immortal Life of Henrietta Lacks,Rebecca Skloot,"Nonfiction,Science,History,Biography,Health,Me...","36,574",4.07
...,...,...,...,...,...,...
8135,10011,Time's Echo,Pamela Hartshorne,"Historical,Historical Fiction,Science Fiction,...",193,3.88
8136,10012,"The Great Good Thing (The Sylvie Cycle, #1)",Roderick Townley,"Fantasy,Fiction,Young Adult,Childrens,Children...",329,3.92
8137,10013,"Edge of Passion (Stealth Guardians, #1)",Tina Folsom,"Fantasy,Paranormal,Romance,Romance,Paranormal ...",102,4.01
8138,10014,Tales from a Not-So-Popular Party Girl (Dork D...,Rachel Renée Russell,"Childrens,Middle Grade,Realistic Fiction,Child...","1,480",4.24


##Model Development dengan content based filtering

In [24]:
data = book_new
data.sample(5)

,id,title,author,genre,review_count,rate
3308,4791,All Aunt Hagar's Children: Stories,Edward P. Jones,"Short Stories,Fiction,Cultural,African America...",271,3.81
4076,5691,Walking His Trail: Signs of God Along the Way,Steve Saint,"Christian,Christian,Christian Living,Biography...",22,4.41
84,99,One Day,David Nicholls,"Fiction,Romance,Contemporary,Womens Fiction,Ch...","18,072",3.80
4191,5818,"Tempestuous (Wondrous Strange, #3)",Lesley Livingston,"Young Adult,Fantasy,Paranormal,Fairies,Romance...",476,4.02
1270,2217,"A Darker Shade of Magic (Shades of Magic, #1)",V.E. Schwab,"Fantasy,Young Adult,Fiction,Fantasy,Magic,Adul...","32,026",4.06


TF-IDF Vectorizer

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
# Inisialisasi TfidfVectorizer
tf = TfidfVectorizer()
 
# Melakukan perhitungan idf pada data genre
tf.fit(data['genre']) 
 
# Mapping array dari fitur index integer ke fitur nama
tf.get_feature_names() 

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['12th',
 '14th',
 '15th',
 '16th',
 '17th',
 '18th',
 '19th',
 '1st',
 '20th',
 '21st',
 '40k',
 'about',
 'abuse',
 'academia',
 'academic',
 'action',
 'activism',
 'adoption',
 'adult',
 'adventure',
 'aeroplanes',
 'africa',
 'african',
 'age',
 'agriculture',
 'albanian',
 'alchemy',
 'alcohol',
 'algeria',
 'aliens',
 'alternate',
 'amateur',
 'amazon',
 'america',
 'american',
 'americana',
 'americans',
 'amish',
 'anarchism',
 'ancient',
 'and',
 'angels',
 'anglo',
 'angola',
 'animal',
 'animals',
 'anime',
 'anthologies',
 'anthropology',
 'anthropomorphic',
 'anti',
 'apocalyptic',
 'apple',
 'archaeology',
 'architecture',
 'art',
 'arthurian',
 'artificial',
 'arts',
 'asia',
 'asian',
 'aspergers',
 'astronomy',
 'atheism',
 'audiobook',
 'australia',
 'authors',
 'autistic',
 'autobiography',
 'aviation',
 'awards',
 'bande',
 'bangladesh',
 'banking',
 'banks',
 'banned',
 'baseball',
 'basketball',
 'batman',
 'bd',
 'bdsm',
 'beast',
 'beauty',
 'belgian',
 'belgiu

Melakukan fit dan transformasi ke dalam bentuk matriks

In [26]:
# Melakukan fit lalu ditransformasikan ke bentuk matrix
tfidf_matrix = tf.fit_transform(data['genre']) 
 
# Melihat ukuran matrix tfidf
tfidf_matrix.shape 

(8140, 706)

In [27]:
# Mengubah vektor tf-idf dalam bentuk matriks dengan fungsi todense()
tfidf_matrix.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [28]:
# Membuat dataframe untuk melihat tf-idf matrix
# Kolom diisi dengan genre buku
# Baris diisi dengan judul buku
 
pd.DataFrame(
    tfidf_matrix.todense(), 
    columns=tf.get_feature_names(),
    index=data.title
).sample(22, axis=1).sample(10, axis=0)

,wonder,business,african,services,google,academic,knitting,novels,sequential,bd,...,psychoanalysis,crafts,ancient,fan,tv,picture,european,theatre,weird,palaeontology
title,,,,,,,,,,,,,,,,,,,,,
"The Husband Hunt (Madison Sisters, #3)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
Hautanummi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.192023,0.0,0.0,0.0
The Hare With Amber Eyes: A Family's Century of Art and Loss,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
The Annotated Archy and Mehitabel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
"The Wind Through the Keyhole (The Dark Tower, #4.5)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
Diary,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.30308,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
Only Ever Yours,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
Intermix Nation,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
"The Emperor of Nihon-Ja (Ranger's Apprentice, #10)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


**Cosine Similarity**

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
 
# Menghitung cosine similarity pada matrix tf-idf
cosine_sim = cosine_similarity(tfidf_matrix) 
cosine_sim

array([[1.        , 0.02986557, 0.07215353, ..., 0.05307196, 0.15989561,
        0.16277007],
       [0.02986557, 1.        , 0.09121858, ..., 0.        , 0.        ,
        0.        ],
       [0.07215353, 0.09121858, 1.        , ..., 0.        , 0.05876795,
        0.01534047],
       ...,
       [0.05307196, 0.        , 0.        , ..., 1.        , 0.        ,
        0.73700145],
       [0.15989561, 0.        , 0.05876795, ..., 0.        , 1.        ,
        0.02569771],
       [0.16277007, 0.        , 0.01534047, ..., 0.73700145, 0.02569771,
        1.        ]])

In [30]:
# Membuat dataframe dari variabel cosine_sim dengan baris dan kolom berupa tilte
cosine_sim_df = pd.DataFrame(cosine_sim, index=data['title'], columns=data['title'])
print('Shape:', cosine_sim_df.shape)
 
# Melihat similarity matrix pada setiap judul buku
cosine_sim_df.sample(5, axis=1).sample(10, axis=0)

Shape: (8140, 8140)


title,The Other Slavery: The Uncovered Story of Indian Enslavement in America,The Absolutely True Diary of a Part-Time Indian,"Die Wellenläufer (Wellenläufer-Trilogie, #1)",Stigma,"Kushiel's Avatar (Phèdre's Trilogy, #3)"
title,,,,,
Predictably Irrational: The Hidden Forces That Shape Our Decisions,0.013372,0.018845,0.000000,0.073768,0.017721
Broken Words: The Abuse of Science and Faith in American Politics,0.089293,0.000000,0.000000,0.000000,0.013480
"Johannes Cabal the Necromancer (Johannes Cabal, #1)",0.000000,0.077661,0.172671,0.013717,0.535241
El día del Watusi,0.000000,0.023932,0.287817,0.012210,0.030809
The Heretic's Daughter,0.047337,0.141354,0.303889,0.020883,0.097004
Vernon Subutex 1,0.000000,0.053379,0.182858,0.014405,0.036349
Code Name: Butterfly,0.000000,0.163258,0.151183,0.083291,0.210172
"Darke (Septimus Heap, #6)",0.000000,0.117570,0.401210,0.040573,0.578885
"Bird Box (Bird Box, #1)",0.000000,0.111273,0.079972,0.205538,0.128006


In [31]:
def book_recommendations(title, similarity_data=cosine_sim_df, items=data[['title', 'genre']], k=5):
    """
    Rekomendasi buku berdasarkan kemiripan dataframe
 
    Parameter:
    ---
    title : tipe data string (str)
                judul buku (index kemiripan dataframe)
    similarity_data : tipe data pd.DataFrame (object)
                      Kesamaan dataframe, simetrik, dengan judul sebagai 
                      indeks dan kolom
    items : tipe data pd.DataFrame (object)
            Mengandung kedua nama dan fitur lainnya yang digunakan untuk mendefinisikan kemiripan
    k : tipe data integer (int)
        Banyaknya jumlah rekomendasi yang diberikan
    ---
 
 
    Pada index ini, kita mengambil k dengan nilai similarity terbesar 
    pada index matrix yang diberikan (i).
    """
 
 
    # Mengambil data dengan menggunakan argpartition untuk melakukan partisi secara tidak langsung sepanjang sumbu yang diberikan    
    # Dataframe diubah menjadi numpy
    # Range(start, stop, step)
    index = similarity_data.loc[:,title].to_numpy().argpartition(
        range(-1, -k, -1))
    
    # Mengambil data dengan similarity terbesar dari index yang ada
    closest = similarity_data.columns[index[-1:-(k+2):-1]]
    
    # Drop title agar titleo yang dicari tidak muncul dalam daftar rekomendasi
    closest = closest.drop(title, errors='ignore')
 
    return pd.DataFrame(closest).merge(items).head(k)

sejumlah rekomendasi buku pada pengguna yang diatur dalam parameter k.

In [32]:
data[data.title.eq('The Great Good Thing (The Sylvie Cycle, #1)')]

,id,title,author,genre,review_count,rate
8136,10012,"The Great Good Thing (The Sylvie Cycle, #1)",Roderick Townley,"Fantasy,Fiction,Young Adult,Childrens,Children...",329,3.92


In [34]:
# Mendapatkan rekomendasi buku yang mirip dengan 'blaze'
book_recommendations('The Great Good Thing (The Sylvie Cycle, #1)')

,title,genre
0,"Tilly and the Lost Fairytales (Pages & Co., #2)","Fantasy,Childrens,Middle Grade,Fiction,Childre..."
1,"Story Thieves (Story Thieves, #1)","Fantasy,Childrens,Middle Grade,Adventure,Ficti..."
2,Steinbeck's Ghost,"Young Adult,Fiction,Childrens,Middle Grade,Mys..."
3,Tilly and the Bookwanderers (Pages & Co. #1),"Childrens,Middle Grade,Fantasy,Writing,Books A..."
4,"Inkheart (Inkworld, #1)","Fantasy,Young Adult,Fiction,Childrens,Middle G..."
